# Sistema de Recomendación de Juegos de Mesa

Este notebook implementa un sistema de recomendación de juegos de mesa basado en **Doc2Vec** y medidas de **similaridad del coseno**. El sistema puede personalizarse para tener en cuenta las valoraciones de los usuarios de los juegos o basarse únicamente en la similaridad textual.



## Cargar Librerías y Definir Funciones
Primero, se cargan las librerías necesarias, como pandas, gensim para trabajar con Doc2Vec, y algunas funciones auxiliares desde un archivo externo (recommender.py).

In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import warnings
import ast
from recommender import *
warnings.filterwarnings('ignore')

## Cargar los Datos
A continuación, cargamos el archivo CSV que contiene la información de los juegos de mesa, incluyendo descripciones, mecánicas, categorías y valoraciones.

In [2]:
# Cargar los datos de juegos (descripciones)
df_juegos = pd.read_csv('../data/raw/boardgames_10000_juegos.csv')

In [3]:
df_juegos.columns

Index(['BGGId', 'Name', 'Year_Published', 'Description', 'Min_Players',
       'Max_Players', 'Min_Playtime', 'Max_Playtime', 'Average_Rating',
       'Bayesian_Average_Rating', 'Number_of_Ratings', 'Mechanics',
       'Categories'],
      dtype='object')

In [4]:
# Obtener la descripción de los juegos
df_descripciones = df_juegos[['BGGId','Name','Description', 'Mechanics', 'Categories', 'Average_Rating', 'Bayesian_Average_Rating']]  

In [5]:
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    10000 non-null  int64  
 1   Name                     10000 non-null  object 
 2   Description              9992 non-null   object 
 3   Mechanics                10000 non-null  object 
 4   Categories               10000 non-null  object 
 5   Average_Rating           10000 non-null  float64
 6   Bayesian_Average_Rating  10000 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 547.0+ KB


In [6]:
df_descripciones.dropna(inplace=True)
df_descripciones.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9992 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   BGGId                    9992 non-null   int64  
 1   Name                     9992 non-null   object 
 2   Description              9992 non-null   object 
 3   Mechanics                9992 non-null   object 
 4   Categories               9992 non-null   object 
 5   Average_Rating           9992 non-null   float64
 6   Bayesian_Average_Rating  9992 non-null   float64
dtypes: float64(2), int64(1), object(4)
memory usage: 624.5+ KB


In [7]:
df_descripciones.sample(5)

,BGGId,Name,Description,Mechanics,Categories,Average_Rating,Bayesian_Average_Rating
590,99553,The Shade of Zombie,The Shade of Zombie is a merger between game o...,['Open Drafting'],['Zombies'],0.00000,0.00000
8571,173334,Venture Hold: A Dungeon Adventure,From publisher blurb:&#10;&#10;Adventure Await...,[],[],0.00000,0.00000
7403,46170,Barony of the Damned,"An adventure set in cursed Mousillon, Bretonni...",[],[],7.55952,6.62624
3907,231497,Wild Water Adrenaline,User Summary&#10;&#10;A kayak racing game.&#10...,[],[],0.00000,0.00000
6441,298201,100 Shadow Lord Kinfolk,From publisher blurb:&#10;&#10;Within this boo...,[],[],0.00000,0.00000


## Preprocesamiento de las Descripciones

Este paso se enfoca en preparar las descripciones de los juegos para que puedan ser usadas en el modelo. Incluye:

- Unir las Descripciones con Mecánicas y Categorías
- Eliminar caracteres especiales.
- Convertir listas de mecánicas y categorías a cadenas de texto.
- Lematizar y limpiar el texto de las descripciones.


In [8]:
# Convertir cadenas de texto que parecen listas a listas reales
df_descripciones['Mechanics'] = df_descripciones['Mechanics'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
df_descripciones['Categories'] = df_descripciones['Categories'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Verificar si las listas están correctamente formateadas
print(df_juegos[['Mechanics', 'Categories']].head())

                                           Mechanics  \
0                  ['Action Points', 'Dice Rolling']   
1                                                 []   
2                                                 []   
3  ['Dice Rolling', 'Grid Movement', 'Hexagon Gri...   
4                                                 []   

                                          Categories  
0  ['Ancient', 'Book', 'Medieval', 'Miniatures', ...  
1                                                 []  
2                                                 []  
3                        ['Wargame', 'World War II']  
4                                                 []  


In [9]:
# Convertir listas a cadenas de texto
df_descripciones['Mecanicas'] = df_descripciones['Mechanics'].apply(lambda x: ', '.join(x) if isinstance(x, list) and len(x) > 0 else "")
df_descripciones['Categorias'] = df_descripciones['Categories'].apply(lambda x: ', '.join(x) if isinstance(x, list) and len(x) > 0 else "")

# Combinar la descripción limpia con las mecánicas y categorías
df_descripciones['Descripcion_completa'] = df_descripciones['Description'] + " " + df_descripciones['Mecanicas'] + " " + df_descripciones['Categorias']

**Preprocesamiento del Texto**
Lematización y eliminación de stopwords utilizando la función `preprocesar_descripcion`.

In [10]:
# Aplicar el preprocesamiento mejorado a las descripciones
df_descripciones['Descripcion_completa'] = df_descripciones['Descripcion_completa'].apply(preprocesar_descripcion)

## Entrenar el Modelo Doc2Vec
En esta sección entrenamos el modelo Doc2Vec utilizando las descripciones completas de los juegos de mesa. Cada juego se representa como un documento, con su texto dividido en palabras (tokens) para ser vectorizado.

In [11]:
# Entrenar Doc2Vec usando los índices del DataFrame como etiquetas
documents = [TaggedDocument(doc.split(), [idx]) for idx, doc in zip(df_descripciones.index, df_descripciones['Descripcion_completa'])]

In [12]:
# Entrenar el modelo Doc2Vec
model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, seed = 42)

## Recomendación Basada en Similaridad de Coseno

En esta parte, se ejecuta una recomendación basada únicamente en la similaridad del coseno entre el prompt del usuario y las descripciones vectorizadas de los juegos de mesa.

In [13]:
# Prompt del usuario
prompt_usuario = "I enjoy strategy games with high replayability and complex decision-making."

In [14]:
# Ejecutar la recomendación usando Doc2Vec
recomendar_juegos_doc2vec(prompt_usuario, df_descripciones, model, top_n=5)

Juegos recomendados:
Leap Frogs! (Similaridad: 0.8136)
Dien Bien Phu (Similaridad: 0.8074)
Chaos Drift (Similaridad: 0.7990)
Tehox Maps A Path Through Giant Trees (Sunset) (Similaridad: 0.7972)
Sparkle Unleashed (Similaridad: 0.7934)


## Recomendación Ponderada (Similaridad y Valoraciones)
Este método tiene en cuenta tanto la similaridad textual como las valoraciones de los usuarios. Al combinar estas dos fuentes de información, se pondera la relevancia del juego para el usuario.

In [15]:
# Ejecutar la recomendación priorizando la similaridad del coseno (peso w = 0.7)
recomendar_juegos_doc2vec_con_valoraciones(prompt_usuario, df_descripciones, model, top_n=5, umbral_similaridad=0.1, w=0.7)

Juegos recomendados (priorizando similaridad del coseno):
Engage: Spaceship Combat (Similaridad ponderada: 0.8489, Similaridad del coseno: 0.8055, Valoración: 9.50)
Amberzine (Issue 11 - 2003) (Similaridad ponderada: 0.8154, Similaridad del coseno: 0.7362, Valoración: 10.00)
A Thousand Enemies (Similaridad ponderada: 0.8145, Similaridad del coseno: 0.7350, Valoración: 10.00)
Alida (Similaridad ponderada: 0.8047, Similaridad del coseno: 0.7209, Valoración: 10.00)
Whirred PLAY (Similaridad ponderada: 0.8031, Similaridad del coseno: 0.7187, Valoración: 10.00)


## Conclusiones
En este notebook, hemos implementado un sistema de recomendación de juegos de mesa basado en:

- Doc2Vec: Para generar representaciones vectoriales de los juegos a partir de su descripción, mecánicas y categorías.
- Similaridad del Coseno: Para medir la cercanía entre el perfil del usuario (prompt) y los juegos disponibles.
- Valoraciones de los Juegos: Opción para ponderar la recomendación con las valoraciones de los usuarios.


Este sistema puede ser fácilmente extendido con nuevas métricas de evaluación o integrarse en una aplicación en tiempo real para recomendaciones personalizadas.